# Agno Agent Demo

This script demonstrates how to create an Agno agent that uses a custom function 
to retrieve context before responding.  In this example, the model will retrieve
current weather conditions in a location and use that information to recommend 
clothing.

The weather API used here is from https://www.weatherapi.com 


In [1]:
# SPDX-FileCopyrightText: (c) 2025 Tenstorrent AI ULC
#
# SPDX-License-Identifier: Apache-2.0
import os

from agno.agent import Agent
from agno.models.openai import OpenAILike
from agno.tools import tool
import requests

### Define function to get current weather conditions

In [2]:
@tool(show_result=True)
def fetch_weather(location):
    params = {
        "q": location,
        "key": os.environ["WEATHER_API_KEY"]
    }
    base_url = "http://api.weatherapi.com/v1/current.json"

    res = requests.get(base_url, params=params)
    data = res.json()
    current_data = data['current']

    temp_string = f"{current_data['temp_f']} °F"
    
    condition_string = current_data['condition']['text']
    location_string = f"{data['location']['name']}, {data['location']['region']}"

    return f"Weather in {location_string}: {temp_string}, {condition_string}.\n"

### Define base URL of endpoint and model id

In [3]:
tt_base_url = os.environ["TT_BASE_URL"]
model_id = "meta-llama/Llama-3.1-8B-Instruct"

### Define the agent
* The `OpenAILike` class from Agno is used as the client, pointing to the Tenstorrent endpoint
* The `fetch_weather` function is passed as an available tool for the agent
* Adjust the instructions to guide the agent's response
* Setting `markdown=True` returns the agents response in markdown formatting

In [4]:
agent = Agent(
    model=OpenAILike(id=model_id, api_key="null", base_url=tt_base_url),
    tools=[fetch_weather],
    instructions=[
        "You are a helpful assistant that gives practical outfit recommendations based on the current weather conditions.",
        "Your only function available is fetch_weather.  Use your existing knowledge for outfit recommendations."
    ],
    markdown=True
)

### Specify the user query and run the agent

In [5]:
user_query = "I'm going to Chicago, how should I dress?"

agent.print_response(user_query, stream=True)

▰▰▰▰▰▰▱ Thinking...
┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ I'm going to Chicago, how should I dress?                                                                       ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Tool Calls ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ • fetch_weather(location=Chicago)                                                                               ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Response (5.9s) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ Weather in Chicago, Illinois: 73.9 °F, Partly Cloudy. Based on the weather in Chicago, I would recommend        ┃
┃ dressing in a way that is suitable for a partly cloudy day with a temperature of 73.9°F.                        ┃
┃                                                                                                                 ┃
┃ Here's a suggested outfit:                                                                                      ┃
┃                                                                                                                 ┃
┃  • A pair of shorts or lightweight pants                                                                        ┃
┃  • A breathable top, such as a t-shirt or a tank top                                                            ┃
┃  • A light jacket or sweater to layer over the top for air-conditioned spaces or cooler evenings                ┃
┃  • Comfortable walking shoes or sneakers                                                                        ┃
┃  • Sunglasses and a hat for sun protection                                                                      ┃
┃                                                                                                                 ┃
┃ This outfit should provide a good balance of comfort and style for a day in Chicago.                            ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛